In [14]:
import random
import numpy as np

# Define frequencies for the C Major scale (in Hz)
c_major_scale = [261.63, 293.66, 329.63, 349.23, 392.00, 440.00, 493.88, 523.25]  # C4 to C5

# Chord progressions in C Major (I, IV, V)
chord_progressions = {
    "I": [261.63, 329.63, 392.00],  # C Major chord
    "IV": [349.23, 440.00, 523.25],  # F Major chord
    "V": [392.00, 493.88, 261.63]    # G Major chord
}

# Durations (in seconds)
durations = [0.25, 0.5, 0.75, 1.0]  # Quarters, halves, etc.

# Generate a music theory-inspired dataset
def generate_music_theory_sequence(scale, chords, num_sequences=1000):
    dataset = []
    for _ in range(num_sequences):
        sequence = []
        for _ in range(random.randint(4, 16)):  # Random sequence length
            if random.random() < 0.6:  # 60% chance of a scale note
                note = random.choice(scale)
                duration = random.choice(durations)
                sequence.append((note, duration))
            else:  # 40% chance of a chord
                chord = random.choice(list(chords.values()))
                duration = random.choice(durations)
                sequence.append((chord, duration))
        dataset.append(sequence)
    return dataset

# Generate the dataset
music_theory_dataset = generate_music_theory_sequence(c_major_scale, chord_progressions)


In [15]:
from pydub import AudioSegment

# Function to generate a square wave for 8-bit sound
def generate_square_wave(frequency, duration, sample_rate=44100, amplitude=0.5):
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    wave = amplitude * np.sign(np.sin(2 * np.pi * frequency * t))
    return wave

# Create the audio file from the generated sequence
def create_8bit_music_clip(sequence, sample_rate=44100):
    music = AudioSegment.silent(duration=0)  # Start with silence
    for element in sequence:
        note, duration = element[0], element[1]
        if isinstance(note, list):  # Handle chords
            chord_wave = sum(generate_square_wave(n, duration, sample_rate) for n in note) / len(note)
            audio = np.int16(chord_wave * 32767)
        else:
            wave = generate_square_wave(note, duration, sample_rate)
            audio = np.int16(wave * 32767)
        
        note_segment = AudioSegment(
            audio.tobytes(), 
            frame_rate=sample_rate, 
            sample_width=2, 
            channels=1
        )
        music += note_segment  # Append each note's segment to the music
    return music

# Example: Create a 10-second clip from a random sequence
clip_duration = 10  # seconds
num_notes = int(clip_duration / random.choice(durations))
generated_clip = create_8bit_music_clip(random.choice(music_theory_dataset)[:num_notes])
generated_clip.export("8bit_music_clip.wav", format="wav")
print("8-bit music clip generated and saved as '8bit_music_clip.wav'")


8-bit music clip generated and saved as '8bit_music_clip.wav'


In [16]:
# Flatten and preprocess dataset for model training
def preprocess_dataset(dataset):
    notes, durations = [], []
    for sequence in dataset:
        for element in sequence:
            note, duration = element[0], element[1]
            if isinstance(note, list):
                notes.extend(note)
            else:
                notes.append(note)
            durations.append(duration)
    return np.array(notes).reshape(-1, 1), np.array(durations).reshape(-1, 1)

notes, durations = preprocess_dataset(music_theory_dataset)

# Normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
notes_scaled = scaler.fit_transform(notes)


In [17]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Prepare data for LSTM
sequence_length = 16
X = []
y = []
for i in range(len(notes_scaled) - sequence_length):
    X.append(notes_scaled[i:i + sequence_length])
    y.append(notes_scaled[i + sequence_length])

X = np.array(X)
y = np.array(y)

# Define the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X, y, epochs=100, batch_size=64)


Epoch 1/100
282/282 [==============================] - 14s 38ms/step - loss: 0.1172
Epoch 2/100
282/282 [==============================] - 11s 39ms/step - loss: 0.1138
Epoch 3/100
282/282 [==============================] - 11s 39ms/step - loss: 0.1134
Epoch 4/100
282/282 [==============================] - 10s 37ms/step - loss: 0.1125
Epoch 5/100
282/282 [==============================] - 10s 35ms/step - loss: 0.1125
Epoch 6/100
282/282 [==============================] - 10s 36ms/step - loss: 0.1123
Epoch 7/100
282/282 [==============================] - 10s 36ms/step - loss: 0.1121
Epoch 8/100
282/282 [==============================] - 10s 36ms/step - loss: 0.1105
Epoch 9/100
282/282 [==============================] - 10s 35ms/step - loss: 0.1028
Epoch 10/100
282/282 [==============================] - 10s 36ms/step - loss: 0.0965
Epoch 11/100
282/282 [==============================] - 10s 35ms/step - loss: 0.0946
Epoch 12/100
282/282 [==============================] - 10s 35ms/step - lo

In [19]:
def generate_music(model, seed_sequence, num_notes, scaler):
    generated_sequence = seed_sequence
    for _ in range(num_notes):
        X_input = generated_sequence[-sequence_length:].reshape(1, sequence_length, 1)
        prediction = model.predict(X_input, verbose=0)
        generated_sequence = np.vstack([generated_sequence, prediction])
    return scaler.inverse_transform(generated_sequence).flatten()

# Generate music
seed_sequence = X[0]
num_notes = 40  # Assuming 20 notes for a 10-second clip
generated_notes = generate_music(model, seed_sequence, num_notes, scaler)

# Convert to 8-bit music clip
generated_clip = create_8bit_music_clip(list(zip(generated_notes, [0.5]*num_notes)))  # Use constant duration for simplicity
generated_clip.export("generated_8bit_music_10sec.wav", format="wav")
print("Generated 20-second 8-bit music clip saved as 'generated_8bit_music_10sec.wav'")


Generated 20-second 8-bit music clip saved as 'generated_8bit_music_10sec.wav'
